In [ ]:
import numpy as np
import cv2
from time import time
from ultralytics import YOLO
from IPython.display import display, Image
from io import BytesIO

class ObjectDetection:

    def __init__(self, capture_index):
        self.capture_index = capture_index
        self.device = 'cpu'
        print("Using Device: ", self.device)
        self.model = self.load_model()
        self.CLASS_NAMES_DICT = self.model.model.names

    def load_model(self):
        model = YOLO(r"C:\Users\mmmel\Desktop\AI_project\best.pt")
        model.fuse()
        return model

    def predict(self, frame):
        results = self.model(frame)
        return results

    def plot_bboxes(self, results, frame):
        for result in results[0]:
            class_id = result.boxes.cls.cpu().numpy().astype(int)
            if class_id == 1:
                confidence = result.boxes.conf.cpu().numpy()
                label = f"{self.CLASS_NAMES_DICT[class_id]} {confidence:0.2f}"
                box = result.boxes.xyxy.cpu().numpy().astype(int)

                # Draw bounding box on the frame
                cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3]), (0, 255, 0), 3)
                cv2.putText(frame, label, (box[0], box[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

        return frame

    def display_image(self, frame):
        _, img_encoded = cv2.imencode('.png', frame)
        display(Image(data=img_encoded.tobytes()))

    def __call__(self):
        cap = cv2.VideoCapture(self.capture_index)
        assert cap.isOpened()
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

        try:
            while True:
                start_time = time()
                ret, frame = cap.read()
                assert ret
                results = self.predict(frame)
                frame = self.plot_bboxes(results, frame)
                end_time = time()
                fps = 1/np.round(end_time - start_time, 2)
                cv2.putText(frame, f'FPS: {int(fps)}', (20, 70), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 2)

                # Display the frame using IPython.display
                self.display_image(frame)
                
        except KeyboardInterrupt:
            # Handle KeyboardInterrupt to stop the video stream gracefully
            pass
        finally:
            cap.release()

detector = ObjectDetection(capture_index=0)
detector()


In [ ]:
pip install playsound

In [ ]:
import numpy as np
import cv2
from time import time
from ultralytics import YOLO
from IPython.display import display, Image
from io import BytesIO
import logging

class ObjectDetection:

    def __init__(self, capture_index=0, model_path="C:\\Users\\mmmel\\Desktop\\AI_project\\best.pt", confidence_threshold=0.5):
        self.capture_index = capture_index
        self.device = 'cpu'
        self.confidence_threshold = confidence_threshold
        logging.info("Using Device: %s", self.device)

        # Load YOLO model
        self.model = self.load_model(model_path)
        self.CLASS_NAMES_DICT = self.model.model.names

    def load_model(self, model_path):
        try:
            model = YOLO(model_path)
            model.fuse()
            return model
        except Exception as e:
            logging.error("Error loading the model: %s", str(e))
            raise

    def predict(self, frame):
        try:
            results = self.model(frame)
            return results
        except Exception as e:
            logging.error("Error during prediction: %s", str(e))
            raise

    def plot_bboxes(self, results, frame):
        for result in results[0]:
            class_id = result.boxes.cls.cpu().numpy().astype(int)
            confidence = result.boxes.conf.cpu().numpy()

            # Adjust this condition based on your class index
            if class_id == 1 and confidence > self.confidence_threshold:
                label = f"{self.CLASS_NAMES_DICT[class_id]} {confidence:0.2f}"
                box = result.boxes.xyxy.cpu().numpy().astype(int)

                # Print bounding box coordinates for debugging
                logging.debug("Bounding Box Coordinates: %s", box)

                # Draw bounding box on the frame
                cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3]), (0, 255, 0), 3)
                cv2.putText(frame, label, (box[0], box[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

        return frame

    def display_image(self, frame):
        _, img_encoded = cv2.imencode('.png', frame)
        display(Image(data=img_encoded.tobytes()))

    def configure_camera(self, cap):
        assert cap.isOpened()
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

    def capture_frames(self):
        cap = cv2.VideoCapture(self.capture_index)
        self.configure_camera(cap)

        try:
            while True:
                start_time = time()
                ret, frame = cap.read()
                assert ret
                results = self.predict(frame)
                frame = self.plot_bboxes(results, frame)
                end_time = time()
                fps = 1 / np.round(end_time - start_time, 2)
                cv2.putText(frame, f'FPS: {int(fps)}', (20, 70), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 2)

                # Display the frame using IPython.display
                self.display_image(frame)

        except KeyboardInterrupt:
            # Handle KeyboardInterrupt to stop the video stream gracefully
            pass
        finally:
            cap.release()

if __name__ == "__main__":
    logging.basicConfig(level=logging.INFO)
    detector = ObjectDetection(capture_index=0)
    detector.capture_frames()


In [ ]:
import numpy as np
import cv2
from time import time, sleep
from ultralytics import YOLO
from IPython.display import display, Image
import tkinter as tk
from playsound import playsound

class ObjectDetection:

    def __init__(self, capture_index):
        self.capture_index = capture_index
        self.device = 'cpu'
        print("Using Device: ", self.device)
        self.model = self.load_model()
        self.CLASS_NAMES_DICT = self.model.model.names

        # GUI setup
        self.root = tk.Tk()
        self.root.title("Fire Detection Alarm")
        self.alarm_label = tk.Label(self.root, text="No Fire Detected", font=("Helvetica", 16))
        self.alarm_label.pack()

    def load_model(self):
        model = YOLO(r"C:\Users\mmmel\Desktop\AI_project\best.pt")
        model.fuse()
        return model

    def predict(self, frame):
        results = self.model(frame)
        #print(results)
        return results

    def plot_bboxes(self, results):
        fire_detected = False

        for result in results[0]:
            class_id = result.boxes.cls.cpu().numpy().astype(int)
            if class_id == 0:
                
                fire_detected = True

        # Update GUI label
        if fire_detected:
            self.alarm_label.config(text="Fire Detected!", fg="red")
            # Play alarm sound
            #playsound(r"C:\Users\mmmel\Desktop\AI_project\fire-alarm-9677.mp3")  # Replace with the actual path to your alarm sound file
        else:
            self.alarm_label.config(text="No Fire Detected", fg="green")

    def display_image(self, frame):
        _, img_encoded = cv2.imencode('.png', frame)
        display(Image(data=img_encoded.tobytes()))

    def update_gui(self):
        self.root.update()

    def __call__(self):
        cap = cv2.VideoCapture(self.capture_index)
        assert cap.isOpened()
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

        try:
            while True:
                start_time = time()
                ret, frame = cap.read()
                assert ret
                results = self.predict(frame)
                self.plot_bboxes(results)
                end_time = time()
                fps = 1/np.round(end_time - start_time, 2)
                cv2.putText(frame, f'FPS: {int(fps)}', (20, 70), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 2)

                # Display the frame using IPython.display
                self.display_image(frame)
                self.update_gui()
                sleep(0.01)  # Small delay to allow GUI to update

        except KeyboardInterrupt:
            print("KeyboardInterrupt: Stopping the video stream and closing the GUI gracefully.")
        finally:
            cap.release()
            self.root.destroy()

if __name__ == "__main__":
    detector = ObjectDetection(capture_index=0)
    detector()


In [ ]:
import numpy as np
import cv2
from time import time, sleep
from ultralytics import YOLO
from IPython.display import display, Image
import tkinter as tk
from playsound import playsound

class ObjectDetection:

    def __init__(self, capture_index):
        self.capture_index = capture_index
        self.device = 'cpu'
        print("Using Device: ", self.device)
        self.model = self.load_model()
        self.CLASS_NAMES_DICT = self.model.model.names

        # GUI setup
        self.root = tk.Tk()
        self.root.title("Fire Detection Alarm")
        self.alarm_label = tk.Label(self.root, text="No Fire Detected", font=("Helvetica", 16))
        self.alarm_label.pack()

    def load_model(self):
        model = YOLO(r"C:\Users\mmmel\Desktop\AI_project\best.pt")
        model.fuse()
        return model

    def predict(self, frame):
        results = self.model(frame)
        #print(results)
        return results

    def plot_bboxes(self, results):
        fire_detected = False

        for result in results[0]:
            class_id = result.boxes.cls.cpu().numpy().astype(int)
            if class_id == 0:
                
                fire_detected = True

        # Update GUI label
        if fire_detected:
            self.alarm_label.config(text="Fire Detected!", fg="red")
            # Play alarm sound
            playsound(r"C:\Users\mmmel\Desktop\AI_project\fire-alarm-9677.mp3")  # Replace with the actual path to your alarm sound file
        else:
            self.alarm_label.config(text="No Fire Detected", fg="green")

    def update_gui(self):
        self.root.update()

    def __call__(self):
        cap = cv2.VideoCapture(self.capture_index)
        assert cap.isOpened()
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

        try:
            while True:
                start_time = time()
                ret, frame = cap.read()
                assert ret
                results = self.predict(frame)
                self.plot_bboxes(results)
                end_time = time()
                fps = 1/np.round(end_time - start_time, 2)
                cv2.putText(frame, f'FPS: {int(fps)}', (20, 70), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 2)

                # Commented out the display_image(frame) line
                #self.display_image(frame)
                self.update_gui()
                sleep(0.01)  # Small delay to allow GUI to update

        except KeyboardInterrupt:
            print("KeyboardInterrupt: Stopping the video stream and closing the GUI gracefully.")
        finally:
            cap.release()
            self.root.destroy()

if __name__ == "__main__":
    detector = ObjectDetection(capture_index=0)
    detector()


In [1]:
import numpy as np
import cv2
from time import time, sleep
from ultralytics import YOLO
from IPython.display import display, Image
import tkinter as tk
from playsound import playsound

class ObjectDetection:

    def __init__(self, capture_index):
        self.capture_index = capture_index
        self.device = 'cpu'
        print("Using Device: ", self.device)
        self.model = self.load_model()
        self.CLASS_NAMES_DICT = self.model.model.names

        # GUI setup
        self.root = tk.Tk()
        self.root.title("Fire Detection Alarm")

        # Alarm control variables
        self.alarm_active = False
        self.consecutive_frames_with_fire = 0
        self.frames_to_trigger_alarm = 5  # Adjust as needed

        # GUI components
        self.alarm_label = tk.Label(self.root, text="No Fire Detected", font=("Helvetica", 16))
        self.alarm_label.pack()

        self.turn_off_alarm_button = tk.Button(self.root, text="Turn Off Alarm", command=self.turn_off_alarm)
        self.turn_off_alarm_button.pack()

    def load_model(self):
        model = YOLO(r"C:\Users\mmmel\Desktop\AI_project\best.pt")
        model.fuse()
        return model

    def predict(self, frame):
        results = self.model(frame)
        # print(results)
        return results

    def plot_bboxes(self, results):
        fire_detected = False

        for result in results[0]:
            class_id = result.boxes.cls.cpu().numpy().astype(int)
            if class_id == 0:
                fire_detected = True

        # Update GUI label
        if fire_detected:
            self.consecutive_frames_with_fire += 1
            if self.consecutive_frames_with_fire >= self.frames_to_trigger_alarm and not self.alarm_active:
                self.trigger_alarm()
        else:
            self.consecutive_frames_with_fire = 0

        if self.alarm_active:
            self.alarm_label.config(text="Fire Detected! Alarm Active", fg="red")
        else:
            self.alarm_label.config(text="No Fire Detected", fg="green")

    def trigger_alarm(self):
        print("Fire detected in at least 5 consecutive frames. Triggering alarm!")
        self.alarm_active = True
        # Play alarm sound
        playsound(r"C:\Users\mmmel\Desktop\AI_project\fire-alarm-9677.mp3")  # Replace with the actual path to your alarm sound file

    def turn_off_alarm(self):
        print("Turning off alarm manually.")
        self.alarm_active = False

    def update_gui(self):
        self.root.update()

    def __call__(self):
        cap = cv2.VideoCapture(self.capture_index)
        assert cap.isOpened()
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

        try:
            while True:
                start_time = time()
                ret, frame = cap.read()
                assert ret
                results = self.predict(frame)
                self.plot_bboxes(results)
                end_time = time()
                fps = 1 / np.round(end_time - start_time, 2)
                cv2.putText(frame, f'FPS: {int(fps)}', (20, 70), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 2)

                # Display the frame using IPython.display (commented out)
                # self.display_image(frame)
                self.update_gui()
                sleep(0.01)  # Small delay to allow GUI to update

        except KeyboardInterrupt:
            print("KeyboardInterrupt: Stopping the video stream and closing the GUI gracefully.")
        finally:
            cap.release()
            self.root.destroy()

if __name__ == "__main__":
    detector = ObjectDetection(capture_index=0)
    detector()


Using Device:  cpu
Model summary (fused): 168 layers, 11126745 parameters, 0 gradients, 28.4 GFLOPs

0: 480x800 1 smoke, 228.4ms
Speed: 5.0ms preprocess, 228.4ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 1 smoke, 221.7ms
Speed: 3.0ms preprocess, 221.7ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 1 default, 1 smoke, 216.0ms
Speed: 4.0ms preprocess, 216.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 1 default, 2 smokes, 198.1ms
Speed: 4.2ms preprocess, 198.1ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 1 default, 1 smoke, 206.9ms
Speed: 5.0ms preprocess, 206.9ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 1 default, 2 smokes, 196.9ms
Speed: 4.5ms preprocess, 196.9ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 1 default, 1 smoke, 211.8ms
Speed: 4.0ms preprocess, 211.8ms inference, 1.0ms po


0: 480x800 1 smoke, 210.0ms
Speed: 3.3ms preprocess, 210.0ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 1 default, 219.9ms
Speed: 4.4ms preprocess, 219.9ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 1 default, 231.0ms
Speed: 3.1ms preprocess, 231.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 2 defaults, 226.8ms
Speed: 4.3ms preprocess, 226.8ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 2 defaults, 225.8ms
Speed: 5.1ms preprocess, 225.8ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 2 defaults, 233.7ms
Speed: 4.2ms preprocess, 233.7ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 1 default, 221.2ms
Speed: 4.3ms preprocess, 221.2ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 1 default, 1 smoke, 226.1ms
Speed: 3.2ms preprocess, 226.1ms inference, 1.0ms postpr

TclError: can't invoke "destroy" command: application has been destroyed